In [ ]:
from hyrun import run

from hyset.v2 import convert_compute_settings
from hyset import create_compute_settings as ccs
from hyif import Xtb
from hyobj import Molecule





## Testing


It is advised to run some tests before running a bigger job or workflow.
In `hyset.v2`, both connection, file transporter, scheduler and database settings can be tested separately.
Additionally, there is the `dry_run` option which creates all the necessary objects but returns without submitting.

### Testing the connection

In [ ]:
from hytools.connection import get_connection
from hyset.base import ComputeSettingsConnection
from dataclasses import asdict
from hyset.v2 import RunSettings
from hyrun.job import get_job
from pathlib import Path

user = 'bodenste'
cs = ComputeSettingsConnection(host='lumi.csc.fi',
                               user=user,
                               port=22,
                               connection_type='ssh',
                               ssh_key_path='~/.ssh/id_rsa_lumi',
                               ssh_socket_path='~/.ssh/lumi.sock')

connection = get_connection(**asdict(cs))
connection.open()
pwd = connection.execute('pwd')
assert pwd.returncode == 0
assert f'/pfs/lustrep2/users/{user}' in pwd.stdout
connection.close()


hyset_1 - ERROR : Memory per CPU not set in compute settings.


### Testing the file transporter

In [22]:


from hytools.file_transporter import get_file_transporter
ft = get_file_transporter(**asdict(cs), dry_run=True)
cmd_send = ft.send(src='test.txt', dest='/scratch/project_465001909/bodenste/')
print(cmd_send)
cmd_receive = ft.receive(src='/scratch/project_465001909/bodenste/test.txt', dest='test.txt')
print(cmd_receive)

rsync -avuzP -e 'ssh -p 22 -i ~/.ssh/id_rsa_lumi -o "ControlPath=~/.ssh/lumi.sock"' test.txt 'bodenste@lumi.csc.fi:/scratch/project_465001909/bodenste/'
rsync -avuzP -e 'ssh -p 22 -i ~/.ssh/id_rsa_lumi -o "ControlPath=~/.ssh/lumi.sock"' 'bodenste@lumi.csc.fi:/scratch/project_465001909/bodenste/test.txt' test.txt/


### Testing the scheduler

In [ ]:
from hyrun.scheduler import get_scheduler
from hyset.base import ComputeSettingsScheduler
from dataclasses import asdict
from hyset.v2 import RunSettings
from hyrun.job import get_job

# slurm scheduler test
cs = ComputeSettingsScheduler(scheduler_type='slurm',
                              slurm_account='test_account',
                              )
scheduler = get_scheduler(**asdict(cs))
rs = RunSettings(work_dir_remote='/tmp/work',
                    data_dir_remote='/tmp/data',
                 submit_dir_remote='/tmp/submit',)
rs.program = 'xtb'
rs.args= ['--gfn', '2']
rs.scheduler = cs
job = get_job(rs)
print(type(job.tasks[0].scheduler), type(scheduler))
cmd = scheduler.gen_job_script(job)
print(cmd)
assert '#SBATCH --account=test_account' in cmd

# local scheduler test

cs = ComputeSettingsScheduler(scheduler_type='local')
scheduler = get_scheduler(**asdict(cs))
rs = RunSettings(work_dir_remote='/tmp/work',
                 data_dir_remote='/tmp/data',
                 submit_dir_remote='/tmp/submit',)
rs.program = 'xtb'
rs.args= ['--gfn', '2']
rs.scheduler = cs
job = get_job(rs)
scmd = scheduler.gen_job_script(job)
assert cmd == 'xtb --gfn 2'


hyset_1 - ERROR : Memory per CPU not set in compute settings.
<class 'hyset.base.scheduler.ComputeSettingsScheduler'> <class 'hyrun.scheduler.slurm.slurm.SlurmScheduler'>
#!/bin/bash
#SBATCH --job-name=hyrun_job
#SBATCH --time=0-00:00:00
#SBATCH --cpus-per-task=1
#SBATCH --ntasks=1
#SBATCH --account=test_account
#SBATCH --output=/tmp/submit/hyrun_job.out
#SBATCH --error=/tmp/submit/hyrun_job.err

set -o errexit

set -x

# Run job no. 0
export SCRATCH="/tmp/work/${SLURM_JOB_ID}"
mkdir -p ${SCRATCH}
cd ${SCRATCH}
 xtb --gfn 2 >> stdout.out 2>> stderr.out


hyset_1 - ERROR : Memory per CPU not set in compute settings.
<class 'hyset.base.scheduler.ComputeSettingsScheduler'> <class 'hyrun.scheduler.local.local.LocalScheduler'>


In [ ]:


# local scheduler test
cs = ComputeSettingsScheduler(scheduler_type='local')
scheduler = get_scheduler(**asdict(cs))
rs = RunSettings()
rs.program = 'xtb'
rs.args= ['--gfn', '2']
job = get_job(rs)
cmd = scheduler.gen_job_script(job)
assert cmd == 'xtb --gfn 2'


In [ ]:
mol = Molecule('O')
cs = ccs()
cs2 = convert_compute_settings(cs)
cs2.database = {'database_name': 'test.db'}
xtb = Xtb(compute_settings=cs2)
rs2 = xtb.setup(mol)



In [ ]:
from hyrun.job import Job
job = Job(tasks=[rs2])
from hytools.file import File
from hydb import get_database
job.job_script = File(name='job.sh', content='ls -l\n')
job.set_hash()
print(job)
db = get_database(job.tasks[0].database)

print(db)
db_id = db.insert_one(job, immutable=['hash', 'db_id'])
job.db_id = db_id
job.status = 'UNKNOWN'
print(f'Inserted job with db_id: {db_id}')
db.update_one(entry=job, db_id=db_id)
print(db[db_id])

dir(db)

In [ ]:
dir(rs2.logger)